In [ ]:
#!/usr/bin/env python

In [ ]:
import wuml 

In [ ]:
data = wuml.wData(xpath='../data/wine.csv', ypath='examples/data/wine_label.csv', label_type='discrete', first_row_is_label=False)

In [ ]:
cf = wuml.classification(data, classifier='NaiveBayes')
print('Running a single classifier')
print(cf)

In [ ]:
print('\n\nRun all regressors sorted by Accuracy')
results = wuml.run_every_classifier(data, y=data.Y, order_by='Test')
Y = results['GP'](data)
print(Y)
print(results['Train/Test Summary'])
import pdb; pdb.set_trace()